# K-Nearest-Neighbors Membership Inference Experiment

In [1]:
import pandas as pd
import sklearn.ensemble as es
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np
import logging

In [2]:
threads = 15

logging.basicConfig()

logger = logging.getLogger('xai-privacy')

In [3]:
from experiment_setup import run_all_experiments
from experiment_setup import get_heart_disease_dataset
from experiment_setup import get_census_dataset

In [4]:
DATASET_HALF = False

data_heart_dict, data_heart_num_dict, data_heart_cat_dict = get_heart_disease_dataset(halve_dataset=DATASET_HALF)
data_census_dict, data_census_num_dict, data_census_cat_dict = get_census_dataset(halve_dataset=DATASET_HALF)

data_heart = data_heart_dict['dataset']
outcome_name_heart = data_heart_dict['outcome']
numeric_features_heart = data_heart_dict['num']

census_num = data_census_num_dict['dataset']
outcome_name_census = data_census_num_dict['outcome']
numeric_features_census = data_census_num_dict['num']

Feature Age: removed 0 rows for missing values.
Feature RestingBP: removed 59 rows for missing values.
Feature Cholesterol: removed 27 rows for missing values.
Feature FastingBS: add unknown category 2.0
Feature RestingECG: add unknown category 3.0
Feature MaxHR: removed 0 rows for missing values.
Feature Oldpeak: removed 7 rows for missing values.
Feature ST_Slope: add unknown category 4.0
Feature CA: add unknown category 4.0
Feature Thal: add unknown category 8.0
Dropped 271 of 1097
Dropped 273 of 1097
Dropped 277 of 1097
Dropped: 2399 of 32561
census: Dropped 3848 of 30162
num: Dropped 19859 of 30162
cat: Dropped 12136 of 30162


This notebook will go through the experiment for membership inference with KNN as an explanation. Membership inference means an attacker with access to the explanation can tell for a sample whether it was included in the training data or not.

The idea for KNN membership inference is as follows: Enter the given sample and check whether the sample is returned as one of its own nearest neighbors. If it is, it is part of the training data. Otherwise it is not.

First, we have to create our own wrapper class for the default scikit-learn KNN classifier. This is necessary because the scikit-learn implementation only returns the indices of the nearest neighbors. However, in order to be useful explanation to a user, the actual feature values of the nearest neighbors need to be returned. This is done by the wrapper class.

In [5]:
# Attack code must be imported so that multiprocessing pool works. Check out knn_attack.py for the implementation of the attack.
from knn_attack import KnnExplainer

Then, we implement the `train_explainer` and `membership_inference_attack_no_model_access` functions:

In [6]:
# Attack code must be imported so that multiprocessing pool works. Check out ice_attack.py for the implementation of the attack.
from knn_attack import KnnMembershipInference

# Dataset 1: Heart Disease

We now generate a KNN explaination for a random sample from the training data as a demonstration of the concept.

In [7]:
features = data_heart.drop(outcome_name_heart, axis=1)
labels = data_heart[outcome_name_heart]

# Train a random forest on training data.
model = es.RandomForestClassifier(random_state=0)
model = model.fit(features, labels)

# Train explainer
exp = KnnExplainer(data_heart, outcome_name_heart)

given_sample = features.sample()
print(f'Given sample: {given_sample.to_numpy()}')

pred = model.predict(given_sample)
print(f'Model prediction: {pred[0]}')

neighbors = exp.explain(given_sample)
print(f'Nearest Neighbors: \n {neighbors.to_numpy()}')

Given sample: [[ 57.    1.    3.  150.  168.    0.    0.  174.    0.    1.6   1.    0.
    3. ]]
Model prediction: 0.0
Nearest Neighbors: 
 [[ 57.    1.    3.  150.  168.    0.    0.  174.    0.    1.6   1.    0.
    3.    0. ]
 [ 59.    1.    4.  140.  177.    0.    0.  162.    1.    0.    1.    1.
    7.    1. ]
 [ 58.    1.    3.  140.  179.    0.    0.  160.    0.    0.    4.    4.
    8.    0. ]
 [ 46.    0.    3.  142.  177.    0.    2.  160.    1.    1.4   3.    0.
    3.    0. ]
 [ 36.    1.    3.  150.  160.    0.    0.  172.    0.    0.    4.    4.
    8.    0. ]]


We will now do a small proof of concept of the experiment with logging enabled to demonstrate how it works.

In [8]:
logger.setLevel(logging.DEBUG)

EXP = KnnMembershipInference(census_num, numeric_features_census, outcome_name_census, random_state=0)
EXP.membership_inference_experiment(num_queries=10, model=DecisionTreeClassifier(random_state=0), model_access=False, threads=1)

logger.setLevel(logging.INFO)

DEBUG:xai-privacy:Numeric Features: ['age', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']
DEBUG:xai-privacy:Categorical Features: []
DEBUG:xai-privacy:Removed 0 test samples due to unknown category.
DEBUG:xai-privacy:[[35 11  0  0 45  0]] taken from training data
DEBUG:xai-privacy:[[   21    10 99999     0    40     1]] taken from test data
DEBUG:xai-privacy:[[   46    16 25236     0    65     1]] taken from training data
DEBUG:xai-privacy:[[17  8  0  0 15  0]] taken from test data
DEBUG:xai-privacy:[[  39   14 7688    0   65    1]] taken from training data
DEBUG:xai-privacy:[[48 10  0  0 65  1]] taken from test data
DEBUG:xai-privacy:[[29 13  0  0 35  0]] taken from training data
DEBUG:xai-privacy:[[   53    14 15024     0    55     1]] taken from test data
DEBUG:xai-privacy:[[55  9  0  0 40  0]] taken from training data
DEBUG:xai-privacy:[[56 10  0  0 41  1]] taken from test data
DEBUG:xai-privacy:Checking sample 0: [35. 11.  0.  0. 45.  0.]
DEBUG:xai-privacy:Inp

Total time: 0.15s (training model: 0.02s, training explainer: 0.00s, experiment: 0.13s)
Accuracy: 1.0, precision: 1.0, recall: 1.0


The proof of concept should show that the membership inference function predicts membership very accurately.

Now we begin executing the actual experiment. We begin by defining the table that will hold the results for all our different experiment variations. Then we execute all variations of the experiment for this dataset. We vary the model between a decision tree, a random forest and a neural network. Each model uses the default configuration of scikit-learn.

In [9]:
dataset_dicts = [data_heart_dict, data_heart_num_dict, data_heart_cat_dict, data_census_dict, data_census_num_dict, data_census_cat_dict]

dt_dict = {'name': 'decision tree', 'model': DecisionTreeClassifier}
rf_dict = {'name': 'random forest', 'model': es.RandomForestClassifier}
nn_dict = {'name': 'neural network', 'model': MLPClassifier}

model_dicts = [dt_dict, rf_dict, nn_dict]

In [10]:
# This will run the experiment for each dataset and model combination

results = run_all_experiments(KnnMembershipInference, dataset_dicts, model_dicts, random_state=0, num_queries=None, model_access=False, threads=threads, repeat=10)

dataset: heart, model: decision tree (repetition 0)
Total time: 4.75s (training model: 0.01s, training explainer: 0.00s, experiment: 4.74s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart, model: decision tree (repetition 1)
Total time: 4.82s (training model: 0.02s, training explainer: 0.00s, experiment: 4.81s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart, model: decision tree (repetition 2)
Total time: 4.83s (training model: 0.00s, training explainer: 0.02s, experiment: 4.81s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart, model: decision tree (repetition 3)
Total time: 4.80s (training model: 0.02s, training explainer: 0.00s, experiment: 4.78s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart, model: decision tree (repetition 4)
Total time: 4.82s (training model: 0.02s, training explainer: 0.00s, experiment: 4.81s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart, model: decision tree (repetition 5)
Total time: 5.02s (training model

c:\users\ml_ks\venvs\xai_new\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Total time: 5.90s (training model: 1.02s, training explainer: 0.00s, experiment: 4.88s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart, model: neural network (repetition 1)


c:\users\ml_ks\venvs\xai_new\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Total time: 5.81s (training model: 0.97s, training explainer: 0.00s, experiment: 4.84s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart, model: neural network (repetition 2)


...

Total time: 55.60s (training model: 19.38s, training explainer: 0.02s, experiment: 36.21s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: census categorical, model: neural network (repetition 8)


c:\users\ml_ks\venvs\xai_new\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Total time: 55.22s (training model: 18.88s, training explainer: 0.02s, experiment: 36.32s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: census categorical, model: neural network (repetition 9)


c:\users\ml_ks\venvs\xai_new\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Total time: 55.50s (training model: 19.11s, training explainer: 0.01s, experiment: 36.38s)
Accuracy: 1.0, precision: 1.0, recall: 1.0


# Results

The results of all variations of the membership inference experiment with KNN. In every experiment, we executed the membership inference attack on each sample of the training data and each sample of the test data. Both datasets are of equal size and originate from the same source dataset.

Accuracy is the percentage of samples whose membership (true or false) was correctly inferred. An algorithm guessing at random would achieve an accuracy of 50 percent.

Precision is the percentage of predicted training samples that is actually in the training data.

Recall is the percentage of training samples whose membership (true) was correctly inferred.

In [11]:
results

,dataset,model,repetition,accuracy,precision,recall
0,heart,decision tree,0,1.0,1.0,1.0
1,heart,decision tree,1,1.0,1.0,1.0
2,heart,decision tree,2,1.0,1.0,1.0
3,heart,decision tree,3,1.0,1.0,1.0
4,heart,decision tree,4,1.0,1.0,1.0
...,...,...,...,...,...,...
175,census categorical,neural network,5,1.0,1.0,1.0
176,census categorical,neural network,6,1.0,1.0,1.0
177,census categorical,neural network,7,1.0,1.0,1.0
178,census categorical,neural network,8,1.0,1.0,1.0


In [12]:
file_name = 'results/1-4-knn-membership-inference-results'
if DATASET_HALF:
    file_name += '_dataset_size_halved'
results.to_csv(file_name + '.csv', index=False, na_rep='NaN', float_format='%.3f')

# Discussion

Just as expected, the attack has an accuracy of 100%. No false positives or false negatives occur.